In [1]:
import pandas as pd
import re
import numpy as np
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

## Loading comments

In [2]:
df_bc = pd.read_csv('data/comments/bitcoin.csv')
df_dj = pd.read_csv('data/comments/dj.csv')
df_ts = pd.read_csv('data/comments/tesla.csv')

## Convert date and delete unnecessary tables

In [3]:
from dateutil import parser
from datetime import date
import re

def convert_date(date_string):
    
    if 'ago' in str(date_string):
        today = date.today()
        return today.strftime("%d-%m-%y")
    
    if 'nan' in str(date_string):
        return
    
    dt = parser.parse(date_string)  
    return dt.strftime("%d-%m-%y")

def convert(df):
    df.drop(['Unnamed: 0'], axis=1, inplace=True)
    df['date'] = df['date'].apply(lambda x: convert_date(x))
    
    return df

In [4]:
df_bc = convert(df_bc)
df_dj = convert(df_dj)
df_ts = convert(df_ts)

C:\Users\cfo19\Anaconda3\lib\site-packages\dateutil\parser\_parser.py:1204: UnknownTimezoneWarning: tzname ET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


In [5]:
del df_ts['+/-']

In [6]:
df_bc = df_bc.dropna(subset=['comment'])
df_dj = df_dj.dropna(subset=['comment'])
df_ts = df_ts.dropna(subset=['comment'])

## Loading price data

In [10]:
df_bcp = pd.read_excel('data/price/BTC_old.xlsx')
df_djp = pd.read_excel('data/price/DJI_old.xlsx')
df_tsp = pd.read_excel('data/price/TSLA_old.xlsx')

,High,Low,Upper Bound,Lower Bound,Date
0,5849.500000,5709.740234,11699.000000,0,04-05-19
1,5901.359863,5596.149902,11802.719726,0,03-05-19
2,5891.899902,5484.779785,11783.799804,0,02-05-19
3,5531.149902,5387.290039,11062.299804,0,01-05-19
4,5417.169922,5346.700195,10834.339844,0,30-04-19
5,5372.229980,5224.640137,10744.459960,0,29-04-19
6,5299.509766,5162.810059,10599.019532,0,28-04-19
7,5313.700195,5218.419922,10627.400390,0,27-04-19
8,5282.970215,5192.450195,10565.940430,0,26-04-19
9,5361.939941,5104.180176,10723.879882,0,25-04-19


 ## Merging price and comment data

In [11]:
df_bc_merged = pd.merge(df_bc,df_bcp, left_on="date", right_on="Date",how="inner")
df_dj_merged = pd.merge(df_dj,df_djp, left_on="date", right_on="Date",how="inner")
df_ts_merged = pd.merge(df_ts,df_tsp, left_on="date", right_on="Date",how="inner")

In [12]:
df_bc_merged = df_bc_merged.dropna(subset=['comment'])
df_dj_merged = df_dj_merged.dropna(subset=['comment'])
df_ts_merged = df_ts_merged.dropna(subset=['comment'])

In [13]:

def add_placeholder(row):
    
    regex1 = r"\b[0-9]{1,}\b"
    regex2 = r"\b[0-9]{1,}\,[0-9]{1,}\b"
    
    #Replace ',' with '.'
    if re.findall(regex2,row['comment']):
        commentwithcomma_list = re.findall(regex2, row['comment'])
        
        for match in commentwithcomma_list:
            match_replaced = match.replace(",",".")
            #print(match_replaced)
            return row['comment'].replace(match,match_replaced)
    #
    if re.findall(regex1,row['comment']):
        predictions_list = re.findall(regex1,row['comment'])
        
        for prediction in predictions_list:    
            predictionVal = float(prediction.replace('$', ''))
    
            if predictionVal < row['Upper Bound'] and predictionVal > row['High']:
                return row['comment'].replace(prediction,' [OPTIMISTICVALUE] ')
            elif predictionVal > row['Lower Bound'] and predictionVal < row['Low']:
                return row['comment'].replace(prediction,' [PESSIMISTICVALUE] ')
            
    return row['comment']

def placeholder(df):
    df.drop(['Date'], axis=1,inplace=True)
    df=df.set_index('date')
    df['updated_comment'] = df.apply(lambda x: add_placeholder(x), axis=1)
    
    return df

In [14]:
df_bc_merged = placeholder(df_bc_merged)
df_dj_merged = placeholder(df_dj_merged)
df_ts_merged = placeholder(df_ts_merged)

In [15]:
df_ts_merged = df_ts_merged.loc[:'02-01-18'] # Alligned data 

## Creating excels for handlabeling

In [16]:
df_bc_merged.to_excel("data/output/bitcoin.xlsx", sheet_name='Sheet_1')
df_dj_merged.to_excel("data/output/dowjones.xlsx", sheet_name='Sheet_1')
df_ts_merged.to_excel("data/output/tesla.xlsx", sheet_name='Sheet_1')

C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://invst.ly/al-zf%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20With%20support%20established%20in%20the%205550.%20area,%20longs%20have%20targets%20of%20resistance%20in%20the%205884.%20and%206357.%20areas.%20Stop%20loss%20in%20the%205320.%20area.%20Shorts%20have%20targets%20of%20support%20in%20the%205320.%20and%205194.%20areas.%20Stop%20loss...Show%20more
' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://invst.ly/alkkj%20%20%20%20With%20support%20in%20the%205320.%20area%20established%20longs%20have%20targets%20of%20resistance%20in%20the%205550.%20and%206357.%20areas.%20Stop%20loss%20in%20the%205194,%20area.%20Shorts%20have%20targets%20of%20support%20in%20the%205194,%20and%205050.%20ar

C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://invst.ly/7w0aj%20My%20take%20on%20where%20we%20are.%20%20%20%20Suspicions%20are%20retrace%20to%206150%20to%206200%20area.%20%20%20However%20if%20we%20retrace%206150%20(blue%20arrow)%20and%20a%204hr%20candle%20closes%20below%206170(cloud%20bottom)%20that%20doesn’t%20support%20a%20bullish%20case....%20I’ve%20also%20noted%20the%204hr%20100%20m...Show%20more
' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://invst.ly/7tqpb%20%20My%20post%20continues%20to%20point%20out%20that%20on%20the%20Weekly%20&amp;%20Daily%20time%20frames%20price%20is%20below%2050%20SMA%20,%20&amp;%20the%20MACD%20the%20zero%20line%20and%20is%20still%20crossed%20down.%20The%20Weekly%20Gung%20Ho%206%20EMA/9EMA%20&amp;%20Daily%20also%20remain%20crossed%2

C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://www.reuters.com/article/uk-hongkong-regulator-cryptocurrencies/hk-regulator-says-to-crackdown-on-crypto-exchanges-for-rule-violations-idUSKBN1FT0AK%20This%20is%20very%20interesting%20as%20a%20lot%20of%20the%20big%20exchanges%20are%20based%20in%20HK
' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://invst.ly/6kkiq.%20.%20I%20know%20this%20is%20BTC%20just%20showing%20how%20alts%20mimic.%20I%20have%20my%20stops%202%20tiers%20the%20bulk%20aggressively%20under%20the%20rising%20trendline%20on%20my%20ETH%20trade%20just%20as%20with%20BTC%20we%20see%20the%20200%20on%20the%20hourly%20(100%20on%202%20hr%20and%2050%20on%204%20hr)%20coming%20-so%20%22%20thin%20...Show%20more
' with link or location/anchor > 255 characters since it

C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://invst.ly/7pcrm%20%20%20%20Price%20on%20the%20Daily%20chart%20is%20currently%20trading%20in%20the%20%20%5bPESSIMISTICVALUE%5d%20-3%20Std%20Deviation%20BB%20channel%20indicating%20strong%20selling%20pressure.%20This%20generally%20last%20%20%20%5bPESSIMISTICVALUE%5d%20-4%20days%20and%20we%20should%20return%20into%20the%201-%20%5bPESSIMISTICVALUE%5d%20%20STD%20Dev%20bands%20(normal%20selling%20pressure)%20but%20it%20c...Show%20more
' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://invst.ly/7od6o%20%20likin'%20these%20short%20time%20frame%20dips%20when%20Gung%20Ho%20%20%5bPESSIMISTICVALUE%5d%20%20hr%204hr%20&amp;%20Daily%20r%20up,%20buy%20n%20sell,%20tight%20stops%20on%20balance%20-%20hate%20being%20long%20when%20I'm%20be

C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://invst.ly/6hfik.%20.%20.%20Sung%20to%20the%20tune%20of%20%22Midnight%20Train%20to%20Georgia%22.%20.%20%20Oooh%20the%20%20%5bPESSIMISTICVALUE%5d%20%20Day%20(SMA)%20%20ooh%20it%20proved%20too%20much%20for%20the%20coin.%20(He%20couldn't%20make%20it).%20So%20he's%20leavin'%20(leavin')%20the%20line%20he's%20come%20to%20know.%20He%20said%20he's%20goin'...(%20he's%20goin'%20Sout...Show%20more
' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Users\cfo19\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:832: UserWarning: Ignoring URL 'https://www.investing.com/analysis/%20%5bPESSIMISTICVALUE%5d%20-reasons-bitcoin-wont-survive-200282110%20this%20article%20contradicts%20a%20lot%20of%20what%20I%20have%20learned%20about%20blockchain%20technology,%20is%20it%20me%20or%20is%20the%20author%20wrong?
' with link or location/anchor > 25

## Reading data after handlabeling

In [23]:
df_aske=pd.read_excel('data/labeled_data/Aske.xlsx')
df_casper=pd.read_excel('data/labeled_data/Casper.xlsx')
df_morten=pd.read_excel('data/labeled_data/Morten.xlsx')
df_mathias=pd.read_excel('data/labeled_data/Mathias.xlsx')



## Concatinating and cleaning data

In [24]:
df_label = pd.concat([df_aske, df_casper, df_morten, df_mathias], ignore_index=True, sort=True)
del df_label['ID']

In [25]:
def clean_data(df, df_type):
    
    if df_type == "big":
        df=df.drop(['comment', 'High','Low','Upper Bound','Lower Bound'], axis=1, inplace=False)
        df.rename(columns={'updated_comment':'COMMENT\n'}, inplace=True)
        df["LABEL"] = np.nan
    
    if df_type == "big" or "hl":
        
        df['COMMENT\n'] = df['COMMENT\n'].apply(lambda x: re.sub('http://\S+|https://\S+', '', x))
        df['COMMENT\n'] = df['COMMENT\n'].apply(lambda x: re.sub("\d+", " ", x))
        df['COMMENT\n'] = df['COMMENT\n'].apply(lambda x: re.sub("\d+", " ", x))
        df.rename(columns={'COMMENT\n':'COMMENT'}, inplace=True)
    
    
    return df

OBS - handlabled data has to be passed with hl whereas the merged original comment data has to be passed with big

In [26]:
df_hlabel = clean_data(df_label,"hl")
df_bc = clean_data(df_bc_merged,"big")
df_dj = clean_data(df_dj_merged,"big")
df_ts =clean_data(df_ts_merged,"big")


This notebook will now

- Tokenize each sentence
- remove stopwords 
- All characters to lower case
- remove puncation 
- Lemmatization with respect to Part of speech tagging

In [27]:
def preperation(input_str, ls):

    input_str = input_str.lower()
    input_str = input_str.translate(str.maketrans("", "", string.punctuation))
    input_str = input_str.strip()
    
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(input_str)
    result = [i for i in tokens if not i in stop_words]
    

    if ls == 'lemma':
        wnl=WordNetLemmatizer()
        lemma=[]
        for word, tag in pos_tag(result):
            if tag.startswith("NN"):
                lemma.append(wnl.lemmatize(word, pos='n'))
            elif tag.startswith('VB'):
                lemma.append(wnl.lemmatize(word, pos='v'))
            elif tag.startswith('JJ'):
                lemma.append(wnl.lemmatize(word, pos='a'))
            else:
                lemma.append(word)
        return lemma
    
    elif ls == 'stemmer':
        stemmer = SnowballStemmer('english')
        stem = []
        for word in result:
            stem.append(stemmer.stem(word))
        return stem
    #for word in result:
        #print(stemmer.stem(word))
    



In [28]:
def preprocess(df):

    df['COMMENT']=df['COMMENT'].apply(lambda x: preperation(x,'lemma'))
    df['COMMENT']=df['COMMENT'].apply(lambda x: ", ".join(x))
    df = df[pd.notnull(df['COMMENT'])]
    
    return df

In [29]:
df_hlabel = preprocess(df_hlabel)

In [30]:
df_bc = preprocess(df_bc)

In [31]:
df_ts = preprocess(df_ts)

In [32]:
df_dj = preprocess(df_dj)

## Output of labeled data to classifier 

In [37]:
df_bc

,COMMENT,LABEL
date,,
04-05-19,"hold, optimisticvalue, becomes, distinct, poss...",NaN
04-05-19,"thumb, nasty, bear, love, get, spank, cryptoland",NaN
04-05-19,"mysterious, crypto, whale, unveils, meteoric, ...",NaN
04-05-19,"probably, pessimisticvalue, monday",NaN
04-05-19,"short, btc, maßsive, profit",NaN
04-05-19,"many, btc, u, buy, da, dip, lot",NaN
04-05-19,"wall, street, analyst, scream, ‘, buy, bitcoin...",NaN
04-05-19,,NaN
04-05-19,"optimisticvalue, resistance, scalper",NaN


In [33]:
df_label.to_excel("data/output/clean_handlabeled_data.xlsx", sheet_name='Sheet_1') 

In [34]:
df_bc.to_excel("data/output/clean_bit_data.xlsx", sheet_name='Sheet_1') 

In [35]:
df_ts.to_excel("data/output/clean_tesla_data.xlsx", sheet_name='Sheet_1') 

In [36]:
df_dj.to_excel("data/output/clean_dowjones_data.xlsx", sheet_name='Sheet_1') 